In [1]:
from sklearn import datasets, model_selection, svm, naive_bayes, tree, metrics, preprocessing
import pandas as pd

In [2]:
train = pd.read_csv('repairedTrain.csv')
test = pd.read_csv('repairedTest.csv')
train.drop('date_of_birth', axis=1, inplace=True)
train.drop('Unnamed: 0', axis=1, inplace=True)
train.drop('Unnamed: 0.1', axis=1, inplace=True)
test.drop('date_of_birth', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0.1', axis=1, inplace=True)

#encoders which didn't work

'''

from sklearn.externals import joblib
train_df = pd.read_csv('repairedTrain.csv')
test_df = pd.read_csv('repairedTest.csv')
train_df.drop('date_of_birth', axis=1, inplace=True)
train_df.drop('Unnamed: 0', axis=1, inplace=True)
train_df.drop('Unnamed: 0.1', axis=1, inplace=True)
test_df.drop('date_of_birth', axis=1, inplace=True)
test_df.drop('Unnamed: 0', axis=1, inplace=True)
test_df.drop('Unnamed: 0.1', axis=1, inplace=True)



train_df['label'] = 'train'
score_df['label'] = 'score'

# Concat
concat_df = pd.concat([train_df , score_df])

# Create your dummies
features_df = pd.get_dummies(concat_df, columns=concat_df.columns, dummy_na=True)

train_df = feature_df[feature_df['label_train'] == 1]
test_df = feature_df[feature_df['label_test'] == 0]
train_df = train_df.drop(['label_train', 'label_test'], axis=1)
test_df = test_df.drop(['label_train', 'label_test'], axis=1)


'''
for column in train.columns:
    if train[column].dtype == type(object):
        le = preprocessing.LabelEncoder()
        train[column] = le.fit_transform(train[column].astype(str))
        test[column] = le.fit_transform(test[column].astype(str))

X = train.drop('class', axis=1)
y = train['class']

X2 = test.drop('class', axis=1)
y2 = test['class']


X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=0)
X_test  =  X_test.fillna(X_test.mean())
train.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,capital-gain,capital-loss,hours-per-week,race,Country,education-status,marital-status,personal_info,class,class number
0,88.0,0,0,0,0,0,0,0,0,0,...,0.0,0,40.0,4,34,15,4,3,0,3734
1,36.0,0,0,0,0,0,0,0,0,0,...,0.0,0,20.0,4,34,11,4,3,0,3516
2,79.0,0,0,0,0,0,0,0,0,0,...,0.0,0,50.0,4,34,9,0,1,0,2331
3,40.0,0,0,0,0,0,0,0,0,0,...,0.0,0,40.0,4,34,9,4,3,0,2960
4,68.0,0,0,0,0,0,0,0,0,0,...,0.0,0,30.0,4,34,11,4,1,0,3424


Na predspracovany tvar datasetu bolo potrebne premenit kategoricke atributy na numericke. Zvolila sa nie najstastnejsia metoda LabelEncoder. Pretoze algoritmy strojoveho ucenia s nimi vykonavaju matematicke operacie, mozu mat mylnu predstavu o tom, ako to naozaj s tymi atributmi funguje. 

# Rozhodovaci strom
Pre hyperparameter tuning sa vybrali nasledujuce parametre:<br>
kriteria rozdelovania (criterion),<br>
maximalna hlbka stromu (max_depth),<br>
minimalny pocet pozorovani v liste (min_samples_leaf).<br>

Pri vyhodnocovani uspesnosti pri vsetkych klasifikatoroch sme si zvolili metriku <b>recall</b>, pretoze nam ovela menej vadi povazovat zdraveho cloveka za choreho ako povazovat choreho za zdraveho. V nasom pripade minoritnou triedou su chori ludia a preto precision moze byt vysoky aj pri trivialnych klasifikatoroch.

In [3]:
cv_params = {'max_depth': list(range(1,15)), 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 4, 8, 12, 16]}
opt = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), cv_params, scoring='recall', cv=5).fit(X_train, y_train)

In [4]:
opt.fit(X, y)
opt.best_params_

{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 1}

Najvacsiu uspesnost malo nastavenie hyperparametrov:<br>
criterion = entropy<br>
max_depth = 8<br>
min_samples_leaf = 1

In [5]:
dtc = tree.DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=1) # kernel, C, gamma
dtc.fit(X, y)

train_preds = dtc.predict(X)
train_accuracy = metrics.accuracy_score(y, train_preds)
print("Vysledky na trenovacich datach\n", train_accuracy)
print(metrics.classification_report(y, train_preds), "\n\nVysledky na testovacich datach")

preds = dtc.predict(X2)
test_accuracy = metrics.accuracy_score(y2, preds)
print(test_accuracy)
print(metrics.classification_report(y2, preds))

Vysledky na trenovacich datach
 1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2628
          1       1.00      1.00      1.00       171

avg / total       1.00      1.00      1.00      2799
 

Vysledky na testovacich datach
0.9845679012345679
             precision    recall  f1-score   support

          0       0.99      0.99      0.99       912
          1       0.88      0.87      0.87        60

avg / total       0.98      0.98      0.98       972



Rozdiely vysledkov pri trenovacich a testovacich dat je minimalny, to znamena ze nedoslo k preuceniu rozhodovacieho stromu<br>

In [6]:
for ith, i in enumerate(dtc.feature_importances_): # cez toto vieme urcit ktore atributy boli nanic a sposobovali akysi sum... mozme ich zahodit
    if i > 0.04:
        print(X_train.columns[ith], i)

TSH 0.05251344790052523
T3 0.6122754943376096
TT4 0.11560724162895379
FTI 0.11571933249130462


Pri natrenovanom rozhodovacom strome s relativne najlepsim nastavenim hyperparametrov boli najdolezistejsie tieto atributy: TSH, T3, TT4 a FTI

# Gaussian Naive Bayes

Pri naivnom bayesovskom klasifikatore niesu ziadne nastavilne hyperparametre ktore stoja za vyhladavanie gridsearchom

In [7]:
gnb = naive_bayes.GaussianNB().fit(X, y)

train_preds2 = gnb.predict(X)
train_accuracy = metrics.accuracy_score(y, train_preds2)
print(train_accuracy)
print(metrics.classification_report(y, train_preds2))

preds2 = gnb.predict(X2)
test_accuracy = metrics.accuracy_score(y2, preds2)
print(test_accuracy)
print(metrics.classification_report(y2, preds2))

pos = 0; neg = 0
for ith, i in enumerate(y_train):
    if train_preds2[ith] == 1:
        pos += 1
    else:
        neg += 1
print("Pocet pozitivne predikovanych:",pos,"\nPocet negativne predikovanychych:",neg)

0.6902465166130761
             precision    recall  f1-score   support

          0       0.99      0.68      0.80      2628
          1       0.15      0.88      0.26       171

avg / total       0.94      0.69      0.77      2799

0.6728395061728395
             precision    recall  f1-score   support

          0       0.99      0.66      0.79       912
          1       0.14      0.85      0.24        60

avg / total       0.93      0.67      0.76       972

Pocet pozitivne predikovanych: 904 
Pocet negativne predikovanychych: 1615


Zrejme kvoli zlemu enkoderu sa algoritmu nedari prave kvoli normalnemu rozdielu z ktoreho vychadza. Skusme pouzit len tie atributy, ktore boli dolezite pri rozvetvovani rozhodovacieho stromu.  

In [8]:
'''
TSH 0.033563168346484366
T3 0.5860878406220137
TT4 0.13802836109356353
FTI 0.07261088742288066
referral source 0.04804579760326189
'''
X3 = X[['TSH', 'T3', 'TT4', 'FTI']]
X4 = X2[['TSH', 'T3', 'TT4', 'FTI']]

gnb = naive_bayes.GaussianNB().fit(X3, y)

train_preds2 = gnb.predict(X3)
train_accuracy = metrics.accuracy_score(y, train_preds2)
print(train_accuracy)
print(metrics.classification_report(y, train_preds2))

preds2 = gnb.predict(X4)
test_accuracy = metrics.accuracy_score(y2, preds2)
print(test_accuracy)
print(metrics.classification_report(y2, preds2))

pos = 0; neg = 0
for ith, i in enumerate(y_train):
    if train_preds2[ith] == 1:
        pos += 1
    else:
        neg += 1
print("Pocet pozitivne predikovanych:",pos,"\nPocet negativne predikovanychych:",neg)

0.939978563772776
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      2628
          1       0.52      0.22      0.31       171

avg / total       0.92      0.94      0.93      2799

0.934156378600823
             precision    recall  f1-score   support

          0       0.95      0.98      0.97       912
          1       0.42      0.18      0.26        60

avg / total       0.92      0.93      0.92       972

Pocet pozitivne predikovanych: 69 
Pocet negativne predikovanychych: 2450


Tu je dokaz toho, ze zle predspracovane atributy viedli k mylnej klasifikacii. Ked pouzijeme iba nasledovne atributy ktore niesu kategoricke, algoritmu sa ovela lepsie pracuje:<br>
TSH 0.033563168346484366<br>
T3 0.5860878406220137<br>
TT4 0.13802836109356353<br>
FTI 0.07261088742288066<br>
Aj tak sa mu vsak nedari najst ludi s chorobami, pretoze nasa zvolena metrika recall ma velmi malu uspesnost a alg. teda nieje vhodny na nas typ klasifikacie s minoritnou triedou pozitivnych chorob

# Support Vector Classifier
Pre hyperparameter tuning sa vybrali parametre:<br>
kernelu - rbf, linear a poly<br> gamma a C (pouzilo by sa viac hodnot, no velmi dlho trvalo prehladavanie a ucenie modelov.<br> Kedze pri SVM ucenie trvalo velmi dlhu dobu, pouzili sa len tie najdolezitejsie atributy  

In [9]:
parameters = [{'kernel': ['rbf', 'poly'],
                   'gamma': [1e-4, 1e-3],
                    'C': [1, 10]},
              {'kernel': ['linear'],
                   'C': [1, 10]}]
clf = model_selection.GridSearchCV(svm.SVC(decision_function_shape='ovr'), parameters, cv=5, scoring = 'recall', n_jobs = -1, verbose = 15 )

In [10]:
clf.fit(X3, y)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print(clf.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    5.5s
[Paralle

0.000 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'poly'}
0.000 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.328 (+/-0.157) for {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'poly'}
0.017 (+/-0.029) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.386 (+/-0.164) for {'C': 10, 'gamma': 0.001, 'kernel': 'poly'}
0.562 (+/-0.142) for {'C': 1, 'kernel': 'linear'}
0.585 (+/-0.121) for {'C': 10, 'kernel': 'linear'}
{'C': 10, 'kernel': 'linear'}


Najlepsie nastavenie hyperparametrov pre SVM klasifikator je s linearnym kernelom a parametrom C=10

In [ ]:
svc = svm.SVC(C=10, kernel='linear') # kernel, C, gamma
svc.fit(X, y)

train_preds = svc.predict(X)
train_accuracy = metrics.accuracy_score(y, train_preds)
print("Vysledky na trenovacich datach\n", train_accuracy)
print(metrics.classification_report(y, train_preds), "\n\nVysledky na testovacich datach")

preds = svc.predict(X2)
test_accuracy = metrics.accuracy_score(y2, preds)
print(test_accuracy)
print(metrics.classification_report(y2, preds))

SVM klasifikator sa nam z casovych dovodov nestihol natrenovat a zistit jeho vysledky....

Na riesenie minoritnych tried by sme mohli vygenerovat viac vzoriek s minoritnou triedou na zvysenie vahy, aby sme tak mohli dopomoct metrike recall

Najlepsie vysledky mal rozhodovaci strom.